# Pulumi 설치 및 Python 샘플 테스트

이 Notebook은 Pulumi를 설치하고, Python을 사용하여 샘플 인프라를 프로비저닝하는 방법을 설명한다. Pulumi는 클라우드 인프라를 코드로 관리할 수 있는 도구이다.

## 1. Pulumi 설치

Pulumi는 다양한 설치 방법을 제공한다. 여기서는 curl을 사용한 설치 방법을 설명한다.

In [1]:
# Linux 또는 macOS 시스템에서는 아래 명령어를 사용하여 Pulumi를 설치할 수 있다.
!curl -fsSL https://get.pulumi.com | sh


설치가 완료되면, Pulumi CLI가 시스템 경로에 추가되어야 한다. 만약 자동으로 추가되지 않았다면, 수동으로 추가해야 한다. 예를 들어, `~/.bashrc` 또는 `~/.zshrc` 파일에 다음을 추가한다:

In [2]:
# Pulumi 경로 추가 예제
PATH=$PATH:~/.pulumi/bin
!source ~/.bashrc

## 2. Pulumi 프로젝트 설정

Pulumi는 프로젝트 단위로 인프라를 관리한다. 프로젝트를 설정하려면 `pulumi new` 명령어를 사용한다. 여기서는 Python 템플릿을 사용한다.

In [3]:
# 프로젝트를 위한 새 디렉터리를 생성하고 이동한다.
!mkdir pulumi-python-demo && cd pulumi-python-demo

# Python 프로젝트를 초기화한다.
!pulumi new python -y

Created project 'pulumi-python-demo'
Successfully created virtual environment!
Installing dependencies...
Finished installing dependencies!

Your new project is ready to go! Follow these steps:

    Run `cd pulumi-python-demo`
    Run `pulumi up` to create your stack
    Run `pulumi stack output` to see the output

More info: https://www.pulumi.com/docs/get-started/python/


위 명령은 새로운 Pulumi 프로젝트를 초기화하고 기본 의존성을 설치한다. `Pulumi.yaml` 파일이 생성되고, 기본적인 Python 스크립트가 포함된다.

## 3. 샘플 인프라 코드 작성

간단한 예제로 AWS EC2 인스턴스를 프로비저닝하는 코드를 작성한다. 이를 위해 `Pulumi.yaml` 파일이 위치한 디렉터리에 `__main__.py` 파일을 생성하고 다음 내용을 추가한다.

In [4]:
%%writefile pulumi-python-demo/__main__.py

import pulumi
import pulumi_aws as aws

# 보안 그룹 생성
group = aws.ec2.SecurityGroup(
    'web-sg',
    description='Enable HTTP access',
    ingress=[
        {
            'protocol': 'tcp',
            'from_port': 80,
            'to_port': 80,
            'cidr_blocks': ['0.0.0.0/0'],
        }
    ],
)

# EC2 인스턴스 생성
server = aws.ec2.Instance(
    'web-server',
    ami='ami-0c55b159cbfafe1f0',  # AWS의 공식 Amazon Linux 2 AMI
    instance_type='t2.micro',
    vpc_security_group_ids=[group.id],
)

# EC2 인스턴스의 IP와 DNS를 출력
pulumi.export('public_ip', server.public_ip)
pulumi.export('public_dns', server.public_dns)


Writing pulumi-python-demo/__main__.py


## 4. Pulumi 스택 배포

이제 `pulumi up` 명령어를 사용하여 인프라를 배포할 수 있다. 이 명령은 Pulumi가 코드를 실행하여 정의된 리소스를 클라우드에 배포하게 한다.

In [5]:
# pulumi 스택 배포
!cd pulumi-python-demo && pulumi up --yes

Previewing update (dev):
     Type                 Name                       Plan
+   pulumi:pulumi:Stack  pulumi-python-demo-dev     create
+   ├─ aws:ec2:SecurityGroup  web-sg                create
+   └─ aws:ec2:Instance       web-server            create

Resources:
    + 3 to create

Updating (dev):
     Type                 Name                       Status
+   pulumi:pulumi:Stack  pulumi-python-demo-dev     created
+   ├─ aws:ec2:SecurityGroup  web-sg                created
+   └─ aws:ec2:Instance       web-server            created

Outputs:
    public_dns: "ec2-xx-xx-xx-xx.compute-1.amazonaws.com"
    public_ip : "xx.xx.xx.xx"

Resources:
    + 3 created

Duration: 45s


배포가 완료되면, 인스턴스의 공용 IP와 DNS를 확인할 수 있다.

## 5. Pulumi 스택 제거

테스트가 완료되면 `pulumi destroy` 명령어를 사용하여 생성된 모든 리소스를 제거할 수 있다.

In [6]:
# pulumi 스택 제거
!cd pulumi-python-demo && pulumi destroy --yes

Previewing destroy (dev):
     Type                 Name                       Plan
-   pulumi:pulumi:Stack  pulumi-python-demo-dev     delete
-   ├─ aws:ec2:Instance       web-server            delete
-   └─ aws:ec2:SecurityGroup  web-sg                delete

Resources:
    - 3 to delete

Destroying (dev):
     Type                 Name                       Status
-   pulumi:pulumi:Stack  pulumi-python-demo-dev     deleted
-   ├─ aws:ec2:Instance       web-server            deleted
-   └─ aws:ec2:SecurityGroup  web-sg                deleted

Resources:
    - 3 deleted

Duration: 30s


이 명령은 Pulumi가 모든 리소스를 제거하고 환경을 초기 상태로 되돌린다.

## 6. 정리

이 Notebook에서는 Pulumi를 설치하고, Python을 사용하여 AWS 인프라를 프로비저닝하는 방법을 다루었다. Pulumi를 사용하면 코드로 클라우드 리소스를 쉽게 관리할 수 있다.